Import stuff
------------

In [1]:
import os
import sys

import numpy as np

import tensorflow as tf
from tensorflow import keras

# Change this with the directory where you cloned the imgdetect-utils repo
basedir = os.path.join(os.path.expanduser('~'), 'Downloads/uv_cleaner/human_detection')
sys.path.append(os.path.join(basedir))
sys.path.append(os.path.join(basedir, 'imgdetect-utils'))

from src.image_helpers import plot_images_grid, create_dataset_files
from src.train_helpers import load_data, plot_results, export_model

# The Tensorflow model and properties file will be stored here
tf_model_dir = os.path.join(basedir, 'models', 'tensorflow')
tf_model_file = os.path.join(tf_model_dir, 'saved_model.pb')
tf_properties_file = os.path.join(tf_model_dir, 'saved_model.json')

# Base directory that contains your training images and dataset files
dataset_base_dir = os.path.join(basedir, 'datasets')
dataset_dir = os.path.join(dataset_base_dir, 'data')

# Store your thermal camera images here
img_dir = os.path.join(dataset_base_dir, 'images')

# Size of the input images
input_size = (24, 32)

Create model directories
------------------------

In [2]:
os.makedirs(tf_model_dir, mode=0o775, exist_ok=True)

Create a dataset files from the available images
------------------------------------------------

In [3]:
dataset_files = create_dataset_files(img_dir, dataset_dir,
                                     split_size=500,
                                     num_threads=1,
                                     resize=input_size,
                                     color_convert='COLOR_BGR2RGB')
dataset_files

Processing 2085 images to 5 dataset files. Format: /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset{:01}.npz
Storing dataset vectors to /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset0.npz
Storing dataset vectors to /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset1.npz
Storing dataset vectors to /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset2.npz
Storing dataset vectors to /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset3.npz
Storing dataset vectors to /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset4.npz


['/Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset0.npz',
 '/Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset1.npz',
 '/Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset2.npz',
 '/Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset3.npz',
 '/Users/hyungjaekim/Downloads/uv_cleaner/human_detection/datasets/data/dataset4.npz']

Get the training and test set randomly out of the dataset with a split of 70/30
-------------------------------------------------------------------------------

In [4]:
train_set, test_set, classes = load_data(*dataset_files, split_percentage=0.8)
print('Loaded {} training images and {} test images. Classes: {}'.format(
    train_set.shape[0], test_set.shape[0], classes))

Loaded 1668 training images and 417 test images. Classes: ['negative' 'positive']


/Users/hyungjaekim/opt/anaconda3/envs/uv_cleaner/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [5]:
train_images = np.asarray([item[0] for item in train_set])
train_labels = np.asarray([item[1] for item in train_set])
test_images = np.asarray([item[0] for item in test_set])
test_labels = np.asarray([item[1] for item in test_set])

Inspect the first 25 images in the training set
-----------------------------------------------


In [6]:
plot_images_grid(images=train_images, labels=train_labels, classes=classes, rows=5, cols=5)


Declare the model
-----------------

* Flatten input
* Layer 1: 60% the number of pixels per image
* Layer 2: 30% the number of pixels per image
* Layer 3: as many neurons as the output labels (in this case 2: negative, positive)

In [7]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=train_images[0].shape),
    keras.layers.Dense(int(0.6 * train_images.shape[1] * train_images.shape[2]), activation=tf.nn.relu),
    keras.layers.Dense(int(0.3 * train_images.shape[1] * train_images.shape[2]), activation=tf.nn.relu),
    keras.layers.Dense(len(classes), activation=tf.nn.softmax)
])

2021-08-09 16:01:09.958500: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-09 16:01:10.003350: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fde1731ad50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-09 16:01:10.003365: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Compile the model
-----------------

- *Loss function*:This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
- *Optimizer*: This is how the model is updated based on the data it sees and its loss function.
- *Metrics*: Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Train the model
---------------

In [9]:
earlystopping = keras.callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

val_images = train_images[:1000]
partial_images = train_images[1000:]
val_labels = train_labels[:1000]
partial_labels = train_labels[1000:]

history = model.fit(partial_images, partial_labels, epochs = 100, 
                    validation_data=(val_images, val_labels), callbacks=[earlystopping])

#model.fit(train_images, train_labels, epochs=100)

Epoch 1/100
21/21 [==============================] - 0s 16ms/step - loss: 1.3188 - accuracy: 0.4970 - val_loss: 0.6223 - val_accuracy: 0.6420
Epoch 2/100
21/21 [==============================] - 0s 10ms/step - loss: 0.5945 - accuracy: 0.6737 - val_loss: 0.4801 - val_accuracy: 0.8510
Epoch 3/100
21/21 [==============================] - 0s 9ms/step - loss: 0.5233 - accuracy: 0.7141 - val_loss: 0.4143 - val_accuracy: 0.8500
Epoch 4/100
21/21 [==============================] - 0s 9ms/step - loss: 0.4060 - accuracy: 0.8473 - val_loss: 0.3623 - val_accuracy: 0.8680
Epoch 5/100
21/21 [==============================] - 0s 8ms/step - loss: 0.3402 - accuracy: 0.8787 - val_loss: 0.3304 - val_accuracy: 0.8580
Epoch 6/100
21/21 [==============================] - 0s 9ms/step - loss: 0.3108 - accuracy: 0.8713 - val_loss: 0.2816 - val_accuracy: 0.8920
Epoch 7/100
21/21 [==============================] - 0s 9ms/step - loss: 0.3491 - accuracy: 0.8458 - val_loss: 0.2748 - val_accuracy: 0.9080
Epoch 8/100

Evaluate accuracy against the test set
--------------------------------------

In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

14/14 [==============================] - 0s 2ms/step - loss: 0.1137 - accuracy: 0.9616
Test accuracy: 0.9616307020187378


Make predictions on the test set
--------------------------------

In [11]:
predictions = model.predict(test_images)
plot_results(images=test_images, labels=test_labels, classes=classes, predictions=predictions, rows=10, cols=4)

Export as a Tensorflow model
----------------------------

In [24]:
"""
export_model(model, tf_model_file,
             properties_file=tf_properties_file,
             classes=classes,
             input_size=(32, 34, 3))
"""

def model_save(model, target, labels=None, overwrite=True):
    import json
    import pathlib

    # Check if we should save it like a .h5/.pb file or as a directory
    model_dir = pathlib.Path(target)
    if str(target).endswith('.h5') or str(target).endswith('.pb'):
        model_dir = model_dir.parent

    # Create the model directory if it doesn't exist
    pathlib.Path(model_dir).mkdir(parents=True, exist_ok=True)

    # Save the Tensorflow model using the .save method
    model.save(target, overwrite=overwrite)

    # Save the label names of your model in a separate JSON file
    if labels:
        labels_file = os.path.join(model_dir, 'labels.json')
        with open(labels_file, 'w') as f:
            f.write(json.dumps(list(labels)))
            
labels = {'negative':0, 'positive':1}
labels.keys()

model_dir = os.path.expanduser('~/Downloads/uv_cleaner/human_detection/models')
model_save(model, model_dir, labels=labels, overwrite=True)

INFO:tensorflow:Assets written to: /Users/hyungjaekim/Downloads/uv_cleaner/human_detection/models/assets


In [22]:
classes

array(['negative', 'positive'], dtype='<U8')